In [71]:
import pandas as pd

PARQUET_PATH = "../data/arxiv_metadata.parquet.gzip"
df = pd.read_parquet(PARQUET_PATH)

import json
with open("categories.json", "r", encoding="utf-8") as f:
    categories = json.load(f)

In [75]:
# import os
# from dotenv import load_dotenv
# from openai import AzureOpenAI

# load_dotenv(".env")


# client = AzureOpenAI(
#     api_key=os.getenv("AZURE_OPENAI_API_KEY"),
#     api_version="2023-12-01-preview",
#     azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
# )

In [76]:
inputs = titles
# save inputs for embedding mapping
# with open("inputs.txt", "w", encoding="utf-8") as f:
#     f.write("\n".join(inputs))

In [77]:
# chunk_size = 999
# embeddings = []
# for i in range(0, len(inputs), chunk_size):
#     chunk = inputs[i:i + chunk_size]
#     embedding = client.embeddings.create(
#         input=chunk,
#         model="text-embedding-3-small",
#     )
#     embeddings.extend(embedding.data)

# # Save in npy format
# import numpy as np

# embeddings_np = np.array([embed.embedding for embed in embeddings])
# np.save('embeddings_cv-10_000.npy', embeddings_np)
# print(embeddings_np.shape)

In [78]:
# import numpy as np
# import pandas as pd
# import plotly.express as px
# from sklearn.manifold import TSNE
# from sklearn.decomposition import PCA

# # Load embeddings and titles
# embeddings = np.load('embeddings.npy')
# tsne = TSNE(n_components=2, random_state=0)
# embeddings_2d = tsne.fit_transform(embeddings)
# tsne_df = pd.DataFrame(embeddings_2d, columns=['x', 'y'])
# tsne_df['title'] = inputs

# # Define a column to specify if the title contains a specific string
# specific_string = "x-ray"  # Change this to your desired string
# tsne_df['highlight'] = tsne_df['title'].apply(lambda x: specific_string in x)

# # Plot with Plotly
# fig = px.scatter(tsne_df, x='x', y='y', color='highlight', hover_data=['title'], title='t-SNE Visualization of Word Embeddings', width=1000, height=1000)
# fig.update_layout(
#     xaxis_title='x',
#     yaxis_title='y',
# )
# fig.show()


In [79]:
# Create title embeddings
import torch
import transformers

device = "cuda"
cache_dir = 'cache'
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)


In [80]:
cfg_roberta = transformers.RobertaConfig.from_pretrained("FacebookAI/roberta-base", cache_dir=cache_dir)
model = transformers.RobertaForMaskedLM.from_pretrained("FacebookAI/roberta-base",config=cfg_roberta, cache_dir=cache_dir).to(device)
tokenizer = transformers.RobertaTokenizer.from_pretrained("FacebookAI/roberta-base", cache_dir=cache_dir)


In [81]:
embeddings = []
with torch.no_grad():
    inputs = tokenizer(inputs, padding=True, truncation=True, return_tensors="pt").to(device)
    x = model.roberta.embeddings(inputs.input_ids)
    embeddings = model.roberta.encoder(x)

embeddings_roberta = embeddings.last_hidden_state.mean(dim=1)

In [89]:
from collections import Counter
from spacy.lang.en import STOP_WORDS

vocab = Counter()
for title in titles:
    vocab.update(title.split())

STOP_WORDS = set(STOP_WORDS)

vocab = {k: v for k, v in vocab.items() if k not in STOP_WORDS}
vocab = sorted(vocab.items(), key=lambda x: x[1], reverse=True)

In [90]:
vocab

[('image', 162),
 ('recognition', 122),
 ('based', 113),
 ('images', 98),
 ('detection', 72),
 ('analysis', 68),
 ('approach', 66),
 ('quantum', 66),
 ('face', 62),
 ('classification', 62),
 ('segmentation', 60),
 ('algorithm', 60),
 ('model', 58),
 ('theory', 53),
 ('learning', 52),
 ('data', 42),
 ('system', 42),
 ('new', 42),
 ('feature', 42),
 ('method', 40),
 ('3d', 40),
 ('field', 37),
 ('automatic', 35),
 ('fusion', 35),
 ('energy', 35),
 ('clustering', 34),
 ('statistical', 32),
 ('linear', 31),
 ('systems', 31),
 ('human', 30),
 ('networks', 29),
 ('local', 29),
 ('visual', 29),
 ('digital', 29),
 ('shape', 29),
 ('wavelet', 28),
 ('phase', 28),
 ('modeling', 27),
 ('models', 27),
 ('sparse', 27),
 ('features', 27),
 ('algorithms', 26),
 ('fast', 26),
 ('robust', 26),
 ('efficient', 26),
 ('neural', 25),
 ('-', 25),
 ('object', 25),
 ('functions', 25),
 ('color', 25),
 ('extraction', 25),
 ('spin', 25),
 ('methods', 24),
 ('network', 24),
 ('handwritten', 24),
 ('vector', 23),

In [91]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.subplots as sp
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

# Load embeddings and titles
embeddings1 = np.load('embeddings.npy')
embeddings2 = np.load('embeddings_cv-10_000.npy')[:1000]
embeddings = np.concatenate([embeddings1, embeddings2])


# Function to generate t-SNE plots
def generate_tsne_plot(embeddings, titles, specific_string, subplot_title):
    tsne = TSNE(n_components=2, random_state=0)
    embeddings_2d = tsne.fit_transform(embeddings)
    tsne_df = pd.DataFrame(embeddings_2d, columns=['x', 'y'])
    tsne_df['title'] = titles
    tsne_df['highlight'] = tsne_df['title'].apply(lambda x: specific_string in x)

    fig = px.scatter(tsne_df, x='x', y='y', color='highlight', hover_data=['title'], title=subplot_title)
    fig.update_layout(xaxis_title='x', yaxis_title='y')
    return fig

# Create subplots
string = "energy"
fig1 = generate_tsne_plot(embeddings, titles, string, "t-SNE Visualization of Word OPENAI")
fig2 = generate_tsne_plot(embeddings_roberta.cpu(), titles, string, "t-SNE Visualization of roBERTa")

fig1.show()
fig2.show()
# # Arrange subplots
# fig = sp.make_subplots(rows=1, cols=2, subplot_titles=["OPENAI", "roBERTa"])
# fig.add_trace(fig1.data[0], row=1, col=1)
# fig.add_trace(fig2.data[0], row=1, col=2)
# fig.update_layout(title_text="t-SNE Visualization of Word Embeddings")
# fig.show()